## Script to create and save and load 3 Synthea bundles:

### 1) Admit to ER 

- Bundle contains Encounter and all referenced resources - no Condition
- Encounter
  - status = in-progress
  - class = EMER
  - should be matched to same encounter with a business identifier but unique id in the Bundle?  Name+"Admit-datetime"
  - for simplicity use the same location and practitioner
  - type is unknown  
- PUT to server to preserve the ID's and references


### 2) Transfer from ER to Inpatient

- Bundle contains updated Encounters and Condition
- Encounter
  - status = in-progress
  - class = IMP
  - should be matched to same encounter with same business identifier and id in the Bundle? Name+"Admit-datetime"
  - for simplicity use the same location and practitioner
  - type is examplotomy
  - hospitalization reason = transfer
- Condition
  - Add Condition for Examplotomy


### 3) Discharge from Inpatient

- Bundle contains updated Encounters
- Encounter
  - status = finished
  - class = IMP
  - should be matched to same encounter with same business identifier and id in the Bundle? Name+"Admit-datetime"
  - for simplicity use the same location and practitioner
  - discharge disposition
  - type is examplotomy - point to same Condition as transfer


### Load to FHIR Reference Server for further testing


- python version 3.6+
- get data from folder
- unbundle
- display

### Issues:

- bad references in Coverage resources in 100 patient bundle.

In [170]:
from os import listdir, path, unlink
from json import load, dumps, loads
from requests import get, post, put, delete
from IPython.display import display, Markdown, HTML
from fhirclient.r4models import bundle as B
from fhirclient.r4models import provenance as P
from fhirclient.r4models import narrative as N
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.fhirreference as FR
import fhirclient.r4models.meta as M
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from datetime import datetime, date
from requests import get, post, put
from uuid import uuid1
from time import sleep
from pprint import pprint
from copy import deepcopy

### globals

In [171]:
#fhir_test_server = 'http://test.fhir.org/r4'
fhir_test_server = 'http://hapi.fhir.org/baseR4'

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json',
    }

types = dict (
Patient = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient',
AllergyIntolerance = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance',
CarePlan = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan',
CareTeam = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam',
Condition = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition',
Device = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-implantable-device',
DiagnosticReport = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab',
                    'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note'],
DocumentReference = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference',
Encounter = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter',
Goal = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal',
Immunization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization',
Location = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-location',
Medication = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication',
MedicationRequest = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest',
Observation = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab',
               'http://hl7.org/fhir/StructureDefinition/vitalsigns',
                'http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus',
               'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height',
               'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age',
               'http://hl7.org/fhir/us/core/StructureDefinition/us-core-pulse-oximetry',],
Organization = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization',
Practitioner = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner',
PractitionerRole = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole',
Procedure = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure',
Provenance = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-provenance',
)


#in_path = '/Users/ehaas/Documents/FHIR/synthea/output/fhir/'
#in_path = '/Users/ehaas/Documents/FHIR/synthea/output/us_core_r4/fhir/'
in_path='test/'
out_path='out/'
#out_path = '/Users/ehaas/Documents/FHIR/ArgoR4Validator/source/examples/'  # append forward slash 
#f_name = 'Denisse335_Stracke611_e760e08d-a0ff-4cd7-83c0-36b27dae6ec3'
#f_name='Alaina222_Gleichner915_0e83bdc3-52c5-4e7e-ba92-174e711ba95c'
#f_name = 'Nadia817_Morissette863_190017e3-417c-46d4-8c0e-dcc9dbc67d25.json'
#f_name = "notify-2.json"
f_name ="notify-100.json"
f_now = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
f_now.as_json()

'2020-05-05T21:01:05.084534Z'

### Get files

In [172]:
def open_file(in_path,f_name): # get files
    with open(f'{in_path}{f_name}',encoding='utf-8', errors='ignore') as f:
        r = f.read()
        return(loads(r))

### First Clear the directory

In [173]:
def clear_path():
    for filename in listdir(out_path):
        file_path = path.join(out_path,filename)
        try:
            if path.isfile(file_path):
               unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(repr(e))

### Write file to examples dir in IG

In [174]:
def write_file(r_json,type,id): # write file  
    with open(f'{out_path}{type}-{id}.json', 'w') as f:
        #print(f'Writing to {out_path}{type}-{id}.json')
        f.write(r_json)

### Fetch Organization from the Bundle to create Provenance resource

In [175]:
def prefetch_org(b):
    #bundle = B.Bundle(b, strict=False)
    #prov_org = (i for i in bundle.entry if i.resource.resource_type =='Organization')
    orgs = (i for i in b['entry'] if i['resource']['resourceType'] =='Organization')
    org_id  = next(orgs)['resource']['id']
    return(org_id)

### Delete Resources ( Teardown )

In [176]:
def delete_r(t,r_id):
    params = dict(
    _cascade='delete',
    )
    r = delete(f'{fhir_test_server}/{t}/{r_id}', params = params, headers = headers)
    return(r)

### Validate Resource Using $validate on Reference Server

In [177]:
def validate(b):
    # note that bundle b is a json  
    r = post(f'{fhir_test_server}/Bundle/$validate', headers = headers, data = b)
    return(r)
    

### Post Transaction Bundle to reference Server

In [178]:
def post_transaction(b):
    # note that bundle b is a json
    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    r = post(fhir_test_server, params = params, headers = headers, data = b)
    return(r)
            

### Create Provenance

- Inputs:
  - what resource is it about
  - Prefetched Organization to use for the author and transmitter ( for now just one will do)
- most everything else is a fixed value.
- validate using the argovalidator  (load to examples)
- add to bundle


In [179]:
def get_agent_type(system,code):
    
    prov_type = CC.CodeableConcept()
    prov_type_coding = C.Coding()
    prov_type_coding.system = system
    prov_type_coding.code = code
    prov_type.coding = [prov_type_coding]
    #print(prov_type)
    return(prov_type)

In [180]:
def get_prov(org,target_type, target_id):
    
  
    who_ref=FR.FHIRReference(dict(
                reference=f'Organization/{org}'
        ))
                             
    prov_target = FR.FHIRReference(dict(
            reference=f'{target_type}/{target_id}'
        ))
    
    f_now = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
    
    prov = P.Provenance()
    prov.id = str(uuid1())
    prov.meta = M.Meta(dict(profile =[types['Provenance']]))
    prov.target= [prov_target]
    prov.recorded = f_now
    prov.agent = []
    
    # this could be a function
    prov_agent = P.ProvenanceAgent()
    prov_agent.who = who_ref
    prov_agent.type = get_agent_type(system='http://hl7.org/fhir/us/core/CodeSystem/us-core-provenance-participant-type',code='transmitter')
    prov.agent.append(prov_agent)
    
    prov_agent = P.ProvenanceAgent()
    prov_agent.who = who_ref
    prov_agent.type = get_agent_type(system='http://terminology.hl7.org/CodeSystem/provenance-participant-type',code='author')
    prov.agent.append(prov_agent)
    
    #print(f'created resource Provenance-{prov.id} for resource {target_type}/{target_id}' )
    
    return(prov)    
    

### Loop through all entries and if US Core type change logical to literal reference and save

#### Loop through entries and create literal to logical reference map

In [181]:
def add_meta(r):
    
    global r_list
                          
    t = r["resourceType"]
    i = r["id"]
    if t in types.keys(): 
       r_list.append(t)
       if t not in ["DiagnosticReport",'Observation']: # add meta profiles
           r['meta']= (dict(profile =[types[t]]))
       elif "Observation" == t:  # "Observation"
           try:
               #['vital-signs','laboratory','72166-2']:
               if 'laboratory' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][0]])
               elif '72166-2' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][2]])
               elif '77606-2' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][3]])
               elif '59576-9' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][4]])
               elif '2708-6' in r['code']['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][5]])
               elif 'vital-signs' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][1]])
               else: # use lab
                   r['meta']= dict(profile =[types[t][0]])
                   pass
           except KeyError:
                   r['meta']= dict(profile =[types[t][0]])

       else:  # "DiagnosticReport"
           try:
               if 'LAB' in r['category'][0]['coding'][0]['code']:
                   r['meta']= dict(profile =[types[t][0]])
               elif r['category'][0]['coding'][0]['code'] in ['LP29684-5','LP29708-2','LP7839-6',]:
                   r['meta']= dict(profile =[types[t][1]])
               else:  # use lab
                   r['meta']= dict(profile =[types[t][0]])
                   pass
           except KeyError:
               r['meta']= dict(profile =[types[t][0]])
                
    '''
    try:
        print(f' update resource {t}-{i} with meta {r["meta"]["profile"][0]}' )
    except Exception as e:
        print(repr(e))
    '''
              
    return(r)
  

### Validate Bundle entries using US Core Validator

In [182]:
def validate_me(bundles):  ### Validate Bundle entries using US Core Validator
    for b in bundles:
        for e in b["entry"]:
                r = e["resource"]
                r_id = r["id"]
                #print(f'id = {r_id}')
                t = r["resourceType"]
                #print(f'type = {t}')
                # convert all logical to literal references
                r_string = dumps(r,indent=4)
                for ref_id, ref_type in ids.items(): 
                    r_string = r_string.replace(f'urn:uuid:{ref_id}',f'{ref_type}/{ref_id}')    
                # validate using the validation build

                write_file(r_string,t,r_id) # validate using the argovalidator (load to examples)
    display(Markdown('**All files loaded onto Validator - run IG-Pub using -d option to validate**'))

## Main File call all the functions above

In [183]:
admit_type = {
             "coding": [
                        {
                            "code": "261665006",
                            "display": "Unknown (qualifier value)",
                            "system": "http://snomed.info/sct"
                        }
                    ],
                    "text": "Unknown (qualifier value)"
                }

td_type =  {
            "coding": [
                {
                    "code": "1234",
                    "display": "Examplotomy Encounter",
                    "system": "http://snomed.info/sct"
                }
            ],
            "text": "Examplotomy Encounter"
        }

transfer_code = dict(
    system = "http://terminology.hl7.org/CodeSystem/admit-source",
    code = "transfer",
    display = "Transfer",
    )

transfer = dict(
       text = "Transfer",
       coding = [transfer_code],
       )

pat_identifier_map = {}
clear_path()
r_list= []
new_b= dict(
    type = "transaction",
    resourceType ="Bundle",
    entry = []
)

synthea_files = [f_name] if f_name else [i for i in listdir(in_path)]

In [184]:
def t_request(t,method,r_id):                   
    req = dict(
        method = method,
        url = f'{t}/{r_id}'
          )
    return(req)

In [185]:
def update_refs(e):

   loc_ref = e['resource']['location'][0]['location']['reference']
   loc_ref = loc_ref.replace("urn:uuid:",'Location/')
   e['resource']['location'][0]['location']['reference']=loc_ref
 
   sub_ref = e['resource']['subject']['reference']
   sub_ref = sub_ref.replace("urn:uuid:",'Patient/')
   e['resource']['subject']['reference'] = sub_ref

   pract_ref = e['resource']['participant'][0]['individual']['reference']
   pract_ref = pract_ref.replace("urn:uuid:",'Practitioner/')
   e['resource']['participant'][0]['individual']['reference'] = pract_ref
    
   org_ref = e['resource']['serviceProvider']['reference']
   org_ref = org_ref.replace("urn:uuid:",'Organization/')
   e['resource']['serviceProvider']['reference'] = org_ref

   return(e)

#### get rid of duplicate entries in bundle ( where fullUrl are the same )

*NOTE THIS Alters the underlying bundle*

In [186]:
def get_dups(entries):
    fullUrls = []
    dups = []
    for i,e in enumerate(entries):
        #print(i, e["fullUrl"])
        if e["fullUrl"] in fullUrls:
            dups.append(i)
        else:
            fullUrls.append(e["fullUrl"])    
    return(dups)

#print(get_dups(b["entry"])[:10])
#print(list(reversed(get_dups(b["entry"])[:10])))


In [187]:
# fix encounter references to same make encounter ID dict      
enc_map = {e["resource"]['subject']['reference']:e["resource"]["id"] 
           for e in b["entry"] if e["resource"]['resourceType'] == "Encounter" 
           and e["resource"]["class"]["code"]=="EMER"}
enc_map

{'urn:uuid:5812ffab-0650-47e1-ae6c-c32d4d0b330d': '9d8984eb-34fc-43de-ac8a-ae8d5d70e754',
 'urn:uuid:8b74fb11-db4d-438d-90d0-a4962a3dcb02': '6be69f5e-b47d-4114-9c20-22b184a70223',
 'urn:uuid:f422d5a4-a267-4d69-9cd2-4f0e8935951b': 'be6005ff-b33b-42cb-a183-e62004697342',
 'urn:uuid:1cfc7b78-44db-4c00-8757-f8025782119c': '3935cb77-9342-4e92-9de5-d62f2958d721',
 'urn:uuid:6f3812b4-1104-4e51-897d-d2d290676f5e': '9ad234ac-c393-4bc6-91ee-4acaf4e65f15',
 'urn:uuid:8b88e3aa-2eee-4a78-8a75-886b7c817982': '559622fc-fdbc-4590-8d77-479c12b6d41c',
 'urn:uuid:b2a96bd2-1478-4558-ac08-b04478bba661': 'a433c568-d8a6-44ec-ba9e-418182fd790f',
 'urn:uuid:710d0935-afa5-4eeb-adb4-6320a7e40baf': '7571ba73-5205-479d-9e04-4ec94652f51a',
 'urn:uuid:79a664e4-a78c-435b-bed2-82a080bbed26': '87e31635-3f6a-40f4-822b-b9b78adf65aa',
 'urn:uuid:d8117431-9327-4cda-8e3f-412bb52a0013': '98e3e96f-b0bc-4f04-85ae-96ecf418ac80',
 'urn:uuid:5a5e3a4d-ee8f-4f47-9f1e-f75a8516c245': 'abda2c73-8a5a-49b5-9f81-830c6a0634a8',
 'urn:uuid

In [188]:
for file in synthea_files: # assume is synthea patient bundle.
    print(file)
    ids = {}
    provs = []
    if file.endswith(".json"):
        print("json file found:\t", file)
        b = open_file(in_path,file)
        
        print(f'bundle count = {len(b["entry"])}')
        # delete duplicates
        for i in reversed(get_dups(b["entry"])):
            try :
                b["entry"].pop(int(i))
            except IndexError:
                print("Index Out of Range")
        print(f'bundle count after depup = {len(b["entry"])}')
        
        admit_b = deepcopy(new_b)
        transfer_b = deepcopy(new_b)
        discharge_b = deepcopy(new_b)
 
        #org_id = prefetch_org(b) # for provenance
        #print(f'Provenance Organization url will be: Organization/{org_id}')
           
        
        for i, e in enumerate(b["entry"]): # fix admit and save identifiers
            r = e["resource"]
            r_id = r["id"]
            t = r["resourceType"]
            e["request"] = t_request(t,"PUT",r_id)
            if t == 'Encounter':
                print(i,r['resourceType'] )
                print(r["class"]["code"])
                try:  # fix admit and add transfer
                    if r['hospitalization']['dischargeDisposition']:
                        print("*"* 10,"DICHARGE","*"* 10 ) 
                except KeyError:
                   print("*"* 10,"ADMIT","*"* 10 ) 
                   print(f'class should be "EMER" is listed as {r["class"]["code"]}')
                   print(f'status should be "in-progress" is listed as {r["status"]}')
                   print(f'type should be "unknown" is listed as {r["type"][0]["text"]}')
                   r['status'] = "in-progress"
                   r["type"] = [admit_type]
                   print(f'class should be "EMER" is listed as {r["class"]["code"]}')
                   print(f'status should be "in-progress" is now  listed as {r["status"]}')
                   print(f'type should be "unknown" is now listed as {r["type"][0]["text"]}')
                   #update identifier and create mapping
                   smush_name = ''.join(r['subject']['display'].split()[1:])
                   smush_date = r['period']['start'].replace('-','').replace(':','')
                   pat_identifier_map[r['subject']['display']] = f"{smush_name}{smush_date}"
                   #print(smush_name, smush_date)
                   enc_identifier = dict(
                       system = "http://example.org",
                       value = pat_identifier_map[r['subject']['display']]
                   )
                   r['identifier']=[enc_identifier]
                   print(f"identifier should be {pat_identifier_map[r['subject']['display']]} is now listed as {r['identifier'][0]['value']}")
                   admit_b['entry'].append(e)
                   print(f'admit bundle length = {len(admit_b["entry"])}')
                   print(f'discharge bundle length = {len(discharge_b["entry"])}')
                   print(f'transfer bundle length = {len(transfer_b["entry"])}....end Admit')


                   # add transfer to new bundle
                   print("*"* 10,"TRANSFER","*"* 10 )
                   t_entry = deepcopy(e)
                   print(f"fullUrl should be { t_entry['fullUrl']}")
                   #t_entry['resource']['id'] = t_uuid
                   t_entry['resource']["class"]["code"] = "IMP"
                   t_entry['resource']["type"] = [td_type]
                   print(f'type should be "examplitis" is now listed as {r["type"][0]["text"]}')
                   t_entry['resource']["hospitalization"] = dict(
                       admitSource = transfer,
                       )

                   print(f"hospitalization should be 'TRANSFER' is now listed as {t_entry['resource']['hospitalization']['admitSource']['text']}")
                   t_entry = update_refs(t_entry)
                   transfer_b['entry'].append(t_entry)
                   print(f'admit bundle length = {len(admit_b["entry"])}')
                   print(f'discharge bundle length = {len(discharge_b["entry"])}')
                   print(f'transfer bundle length = {len(transfer_b["entry"])}....end Transfer')


                else: # update discharge and remove to discharge bundle
                    #update the id
                    for k,v in enc_map.items():
                        if r['subject']['reference'] == k:
                            print(f"discharge matches patients {k}")
                            r['id'] = v
                            e['fullUrl'] = f"urn:uuid:{v}"
                            e["request"] = t_request(t,"PUT",v)
                            print(f"discharge matches patients {k} id = {r['id']} and fullUrl = {e['fullUrl']}")
                    print(f'status should be "finished" is listed as {r["status"]}')
                    print(f'type should be "examplitis" is listed as {r["type"][0]["text"]}')
                    print(f'class should be "IMP" is listed as {r["class"]["code"]}')
                    r['status'] = "finished"
                    r["type"] = [td_type]
                    r["class"]["code"] = "IMP"
                    print(f'status should be "finished" is now listed as {r["status"]}')
                    print(f'type should be "examplitis" is now listed as {r["type"][0]["text"]}')
                    print(f'class should be "IMP" is listed as {r["class"]["code"]}')
                    #update identifier and create mapping
                    smush_name = ''.join(r['subject']['display'].split()[1:])
                    smush_date = r['period']['start'].replace('-','').replace(':','')
                    pat_identifier_map[r['subject']['display']] = f"{smush_name}{smush_date}"
                    enc_identifier = dict(
                               system = "http://example.org",
                               value = pat_identifier_map[r['subject']['display']]
                           )
                    r['identifier']=[enc_identifier]
                    print(f"identifier should be {pat_identifier_map[r['subject']['display']]} is now listed as {r['identifier'][0]['value']}")
                    entry = update_refs(e)
                    discharge_b['entry'].append(e)
                    print(f'admit bundle length = {len(admit_b["entry"])}')
                    print(f'discharge bundle length = {len(discharge_b["entry"])}')
                    print(f'transfer bundle length = {len(transfer_b["entry"])}....end Discharge')

                # remove condition from admit bundle and add to discharge bundle              
            elif t == 'Condition':
               for k,v in enc_map.items():  # Update references
                    if r['subject']['reference'] == k:
                        r['subject']['reference'] = k.replace("urn:uuid:",'Patient/')
                        r['encounter']['reference'] = f"urn:uuid:{v}"
               transfer_b['entry'].append(e)
               print(f'transfer bundle length = {len(transfer_b["entry"])}....end add Condition')
            else:
                admit_b['entry'].append(e)
                print(f'admit bundle length = {len(admit_b["entry"])}....end add else = {t}')
            
               
            '''
            if t in types.keys(): # see what types are in bundle
               r_list.append(t)
            # get a list of id's to create literal references later using a string replace 
            ids[r_id]=t
            
            #e['resource']= add_meta(r) # add meta elements to resource  - Synthea already does this!!!
            
            #change POST to PUT and update the URL
            e['request']['method']='PUT'
            e['request']['url']=f'{e["request"]["url"]}/{r_id}'
            
            # create provenance and add to list
            provs.append(get_prov(org_id,t,r_id)) #create corresponding Provenance resource and add to list
            '''
            
        '''
        # create a transaction bundle provenance ,stick with dict type
        pb = dict(
            resourceType="Bundle",
            type="transaction",
            )
        
        pb['entry'] = []
        for p in provs:
            ids[p.id]='Provenance'
            
            e = dict( # no fullURL for now - use relative literal references
                resource = p.as_json(),
                request = dict(
                    method='PUT',
                    url= f'Provenance/{p.id}',
                    ), 
                )
            
            pb['entry'].append(e)
            
          
        '''
        
        #write bundles to temp file to test manually
        #for i, bundle in enumerate([pb,b]):
        #print(len(b['entry']),len(new_b),len(b['entry'] + new_b))
        
        for use in (['admit','transfer','discharge']):
            d_bundle = eval(f'{use}_b')
            #list_FullUrls = [(e['resource']['resourceType'],e['fullUrl']) for e in d_bundle['entry']]
            #pprint(list_FullUrls)
            #pprint([i for i, x in enumerate(list_FullUrls) if list_FullUrls.count(x) > 1])
            j_bundle = dumps(d_bundle, indent=4)
            with open(f'{out_path}/{use}_{f_name}', 'w') as f:
                f.write(j_bundle)
         
        
            print(f'*********validate {use} bundle using the validation build ********')
            r = validate(j_bundle)
            display(HTML('<h1>Request Headers</h1>'))
            pprint(r.request.headers)
            display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))


            print(f'*********load {use} bundle to FHIR Server ***********')
            r = post_transaction(j_bundle)
            display(HTML('<h1>Request Headers</h1>'))
            pprint(r.request.headers)
            #display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))
            display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3>'))
            pprint(r.json())
            sleep(5)
            
         
            '''print(f'*********Optional delete {use} bundle using the validation build ********')
            for e in d_bundle['entry']:
                t = e['resource']['resourceType']
                r_id = e['resource']['id']
                #print(t, r_id)
                r = delete_r(t,r_id)
                display(HTML('<h1>Request Headers</h1>'))
                pprint(r.request.headers)
                #display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))
                display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3>'))
                pprint(r.json())
           '''

print('======Fin======')

notify-100.json
json file found:	 notify-100.json
bundle count = 868
bundle count after depup = 757
0 Encounter
EMER
********** DICHARGE **********
discharge matches patients urn:uuid:5812ffab-0650-47e1-ae6c-c32d4d0b330d
discharge matches patients urn:uuid:5812ffab-0650-47e1-ae6c-c32d4d0b330d id = 9d8984eb-34fc-43de-ac8a-ae8d5d70e754 and fullUrl = urn:uuid:9d8984eb-34fc-43de-ac8a-ae8d5d70e754
status should be "finished" is listed as in-progress
type should be "examplitis" is listed as Unknown (qualifier value)
class should be "IMP" is listed as EMER
status should be "finished" is now listed as finished
type should be "examplitis" is now listed as Examplotomy Encounter
class should be "IMP" is listed as IMP
identifier should be Adrienne302Hansen12120190723T0308300700 is now listed as Adrienne302Hansen12120190723T0308300700
admit bundle length = 0
discharge bundle length = 1
transfer bundle length = 0....end Discharge
admit bundle length = 1....end add else = Patient
admit bundle length 

{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/fhir+json', 'Connection': 'keep-alive', 'Content-Type': 'application/fhir+json', 'Content-Length': '1549069'}


WARNING,"[Bundle.entry[0].resource.identifier[2].type, Line 138, Col 40]","None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/identifier-type (http://hl7.org/fhir/ValueSet/identifier-type, and a code should come from this value set unless it has no suitable code) (codes = http://terminology.hl7.org/CodeSystem/v2-0203#SS)"
INFORMATION,"[Bundle.entry[0].resource.extension[2], Line 53, Col 22]",This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
INFORMATION,"[Bundle.entry[0].resource.extension[4], Line 61, Col 22]",This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
WARNING,"[Bundle.entry[0].resource.communication[0].language, Line 104, Col 40]","None of the codes provided are in the value set http://hl7.org/fhir/us/core/ValueSet/simple-language (http://hl7.org/fhir/us/core/ValueSet/simple-language, and a code should come from this value set unless it has no suitable code) (codes = urn:ietf:bcp:47#en-US)"
WARNING,"[Bundle.entry[1].resource, Line 216, Col 26]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[2].resource, Line 229, Col 62]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[3].resource, Line 285, Col 30]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[4].resource, Line 307, Col 26]",dom-6: A resource should have narrative for robust management [text.div.exists()]
INFORMATION,"[Bundle.entry[4].resource.identifier[0], Line 325, Col 22]",This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization
WARNING,"[Bundle.entry[5].resource, Line 360, Col 26]",dom-6: A resource should have narrative for robust management [text.div.exists()]
WARNING,"[Bundle.entry[6].resource, Line 459, Col 29]",dom-6: A resource should have narrative for robust management [text.div.exists()]


*********load admit bundle to FHIR Server ***********


{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/fhir+json', 'Connection': 'keep-alive', 'Content-Type': 'application/fhir+json', 'Content-Length': '1549069'}


...

{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/fhir+json', 'Connection': 'keep-alive', 'Content-Type': 'application/fhir+json', 'Content-Length': '303277'}


WARNING,"[Bundle.entry[0].resource, Line 65, Col 29]",dom-6: A resource should have narrative for robust management [text.div.exists()]
ERROR,"[Bundle.entry[0].resource.type[0].coding[0], Line 72, Col 30]","Unknown code {http://snomed.info/sct}1234 for ""http://snomed.info/sct#1234"""
WARNING,"[Bundle.entry[0].resource.type[0], Line 70, Col 22]",Error Invalid filter criteria - code does not exist: {http://snomed.info/sct}308335008 validating CodeableConcept
WARNING,"[Bundle.entry[1].resource, Line 144, Col 29]",dom-6: A resource should have narrative for robust management [text.div.exists()]
ERROR,"[Bundle.entry[1].resource.type[0].coding[0], Line 151, Col 30]","Unknown code {http://snomed.info/sct}1234 for ""http://snomed.info/sct#1234"""
WARNING,"[Bundle.entry[1].resource.type[0], Line 149, Col 22]",Error Invalid filter criteria - code does not exist: {http://snomed.info/sct}308335008 validating CodeableConcept
WARNING,"[Bundle.entry[2].resource, Line 223, Col 29]",dom-6: A resource should have narrative for robust management [text.div.exists()]
ERROR,"[Bundle.entry[2].resource.type[0].coding[0], Line 230, Col 30]","Unknown code {http://snomed.info/sct}1234 for ""http://snomed.info/sct#1234"""
WARNING,"[Bundle.entry[2].resource.type[0], Line 228, Col 22]",Error Invalid filter criteria - code does not exist: {http://snomed.info/sct}308335008 validating CodeableConcept
WARNING,"[Bundle.entry[3].resource, Line 302, Col 29]",dom-6: A resource should have narrative for robust management [text.div.exists()]
ERROR,"[Bundle.entry[3].resource.type[0].coding[0], Line 309, Col 30]","Unknown code {http://snomed.info/sct}1234 for ""http://snomed.info/sct#1234"""


*********load discharge bundle to FHIR Server ***********


{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/fhir+json', 'Connection': 'keep-alive', 'Content-Type': 'application/fhir+json', 'Content-Length': '303277'}


{'issue': [{'code': 'processing',
            'diagnostics': 'Resource '
                           'Patient/f7f38797-25f9-4df8-842a-186b4091e32a not '
                           'found, specified in path: Encounter.subject',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome',
 'text': {'div': '<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation '
                 'Outcome</h1><table border="0"><tr><td style="font-weight: '
                 'bold;">ERROR</td><td>[]</td><td><pre>Resource '
                 'Patient/f7f38797-25f9-4df8-842a-186b4091e32a not found, '
                 'specified in path: Encounter.subject</pre></td>\n'
                 '\t\t\t\t\t\n'
                 '\t\t\t\t\n'
                 '\t\t\t</tr>\n'
                 '\t\t</table>\n'
                 '\t</div>',
          'status': 'generated'}}
======Fin======
